In [1]:
import sys
import pathlib
pth=str(pathlib.Path().absolute())
sys.path.append(('\\').join(pth.split('\\')[:-1])+"\\Utils")
from Utilities import *
from MF import *
from Visualization_utilities import *
from transformation import Transformation
from Signatures import Signature
import os
import open3d as o3d

In [20]:
def project_pcl_pca(pcl,voxel_size=0.08,nb=5,std_ratio=1.5,p_th=0.5,ig="sphericity",upper=True,scaling_factor=1):
    #inverser scaling
    pcl.scale(scaling_factor,center=pcl.get_center())
    
    
    
    #o3d.visualization.draw_geometries([pcl])
    pcl.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(30))
    pcl, ind = pcl.remove_statistical_outlier(nb_neighbors=nb,std_ratio=std_ratio)
    pcl=pcl.voxel_down_sample(voxel_size=voxel_size)
    pcl.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(30))
    #o3d.visualization.draw_geometries([pcl])

    indexes=np.arange(np.asarray(pcl.points).shape[0])
    pt = o3d.geometry.KDTreeFlann(pcl)
    signatures=Signature(pcl,pt,indexes)
    Signature().v_build(signatures,NN_Criteria="KNN",
                    rad=np.std(np.asarray(pcl.points)-np.mean(np.asarray(pcl.points),axis=0))/8,
                   NN=30)

    signaturesp=remove_discontinuities_by_curvature(pcl,signatures,pt,curvature=0.4,radius=0.5)
    #graficarPropiedad(signaturesp,pcl,property=ig,frac=1)

    #LESS THAN
    pp=np.vectorize(lambda signature,att: getattr(signature,att))(signaturesp,ig)
    if upper:
        pp=np.delete(signaturesp,pp>p_th)
    else:
        pp=np.delete(signaturesp,pp<p_th)
    points=np.vectorize(lambda x: x.pointCoordinates,signature="()->(j)")(pp)
    
    data_mean = np.mean(points, axis=0) #Calculate the average value of the column
    # Normalized 
    normalize_data = points - data_mean
    # SVD decomposition
    # Construct covariance matrix
    H = np.dot(normalize_data.T, normalize_data)
    # SVD decomposition
    eigenvectors, eigenvalues, eigenvectors_t = np.linalg.svd(H)   # H = U S V
    # Reverse order
    sort = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[sort]
    eigenvectors = eigenvectors[:, sort]
    return eigenvectors[:, 0],eigenvectors[:, 1]

In [11]:
data_dir="D:\\Documentos\\INNOVATE\\Base_de_datos\\BEAMS\\Aceptado"
pcls=os.listdir(data_dir)
os.path.join(data_dir,pcls[0])
_dir="D:\\Documentos\\INNOVATE\\Base_de_datos\\Debug\\seccion.pts"


In [23]:
pts = 'Beam_L\\A1_MODELO_3D_revA_L3x3x1_400_pulg_005.pts'
for pcl in pcls:
    print(pcl)
    dpf=o3d.io.read_point_cloud(os.path.join(data_dir,pcl))
    NT = len(dpf.points)

    if NT >= 4096:
        dpf = dpf.uniform_down_sample(int(NT/4096))
    
    dpf.translate(-dpf.get_center())
    
    maxp=np.max(np.hstack((dpf.get_min_bound(),dpf.get_max_bound())))
    dpf.scale(1/maxp,center=dpf.get_center())
    #o3d.visualization.draw_geometries([dpf])

    ev1,ev2=project_pcl_pca(dpf,
             voxel_size=0.02,
             nb=5,
             std_ratio=1.5,
             p_th=0.1,
             ig="Linearity",
             upper=True,
            scaling_factor=maxp
            )

#o3d.visualization.draw_geometries([dpf])

A1_MODELO_3D_revA_C3X600_pulg_004.pts
A1_MODELO_3D_revA_C3X600_pulg_005.pts
A1_MODELO_3D_revA_C3X600_pulg_006.pts
A1_MODELO_3D_revA_C3X600_pulg_007.pts
A1_MODELO_3D_revA_C3X600_pulg_008.pts
A1_MODELO_3D_revA_C3X600_pulg_009.pts
A1_MODELO_3D_revA_C3X601_pulg_013.pts
A1_MODELO_3D_revA_C3X601_pulg_014.pts
A1_MODELO_3D_revA_C3X601_pulg_015.pts
A1_MODELO_3D_revA_C3X601_pulg_018.pts
A1_MODELO_3D_revA_C3X602_pulg_022.pts
A1_MODELO_3D_revA_C3X602_pulg_024.pts
A1_MODELO_3D_revA_C3X602_pulg_025.pts
A1_MODELO_3D_revA_C4x4_500_pulg_008.pts
A1_MODELO_3D_revA_C4x4__pulg_4_5.pts
A1_MODELO_3D_revA_C4x5_401_pulg_018.pts
A1_MODELO_3D_revA_C4x5_401_pulg_019.pts
A1_MODELO_3D_revA_C4x5__pulg_5_4.pts
A1_MODELO_3D_revA_C5x6_700_pulg_004.pts
A1_MODELO_3D_revA_C5x900_pulg_001.pts
A1_MODELO_3D_revA_C5x_pulg_5x9.pts
A1_MODELO_3D_revA_C6x10__pulg_0_5.pts
A1_MODELO_3D_revA_C6x1300_pulg_001.pts
A1_MODELO_3D_revA_C6x1300_pulg_004.pts
A1_MODELO_3D_revA_C6x1_pulg_x13.pts
A1_MODELO_3D_revA_C6x8_200_pulg_001.pts
A1_MODE

N200A_1_2_ESTRUCTURA_EXTERIOR_Rev0_291020_ACIS__mod_29_04_W10x3000_pulg_004.pts
N200A_1_2_ESTRUCTURA_EXTERIOR_Rev0_291020_ACIS__mod_29_04_W10x3_pulg_x30.pts
N200A_1_2_ESTRUCTURA_EXTERIOR_Rev0_291020_ACIS__mod_29_04_W12x3000_pulg_005.pts
N200A_1_2_ESTRUCTURA_EXTERIOR_Rev0_291020_ACIS__mod_29_04_W12x3000_pulg_006.pts
N200A_1_2_ESTRUCTURA_EXTERIOR_Rev0_291020_ACIS__mod_29_04_W12x3000_pulg_008.pts
N200A_1_2_ESTRUCTURA_EXTERIOR_Rev0_291020_ACIS__mod_29_04_W12x4000_pulg_003.pts
N200A_1_2_ESTRUCTURA_EXTERIOR_Rev0_291020_ACIS__mod_29_04_W12x4001_pulg_010.pts
N200A_1_2_ESTRUCTURA_EXTERIOR_Rev0_291020_ACIS__mod_29_04_W12x4001_pulg_013.pts
N200A_1_2_ESTRUCTURA_EXTERIOR_Rev0_291020_ACIS__mod_29_04_W12x4001_pulg_018.pts
N200A_1_2_ESTRUCTURA_EXTERIOR_Rev0_291020_ACIS__mod_29_04_W12x4003_pulg_033.pts
N200A_1_2_ESTRUCTURA_EXTERIOR_Rev0_291020_ACIS__mod_29_04_W12x4003_pulg_034.pts
N200A_1_2_ESTRUCTURA_EXTERIOR_Rev0_291020_ACIS__mod_29_04_W12x4004_pulg_042.pts
N200A_1_2_ESTRUCTURA_EXTERIOR_Rev0_291020_A

N300A0_Modelo3D_Civil__Estructural_Rev1_W12x3501_pulg_011.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W12x3501_pulg_012.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W12x3501_pulg_013.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W12x3_pulg_x35.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W12x4000_pulg_001.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W12x4000_pulg_002.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W12x4000_pulg_003.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W12x4000_pulg_005.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W12x4000_pulg_006.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W12x4000_pulg_007.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W12x4000_pulg_008.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W12x4001_pulg_010.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W12x4001_pulg_012.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W12x4001_pulg_013.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W12x4001_pulg_014.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W12x4001_pulg_015.pts
N300A0_Mode

N300A0_Modelo3D_Civil__Estructural_Rev1_W8x3101_pulg_018.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W8x3101_pulg_019.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W8x3102_pulg_021.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W8x3102_pulg_022.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W8x3102_pulg_024.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W8x3102_pulg_028.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W8x3103_pulg_031.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W8x3103_pulg_032.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W8x3103_pulg_034.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W8x3103_pulg_035.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W8x3103_pulg_036.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W8x3103_pulg_039.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W8x3104_pulg_045.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W8x3104_pulg_046.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W8x3104_pulg_047.pts
N300A0_Modelo3D_Civil__Estructural_Rev1_W8x3104_pulg_048.pts
N300A0_Modelo3D_Civil__E

In [30]:
dpf=o3d.io.read_point_cloud(_dir)
o3d.visualization.draw_geometries([dpf])
NT = len(dpf.points)

if NT >= 4096:
    dpf = dpf.uniform_down_sample(int(NT/4096))
    
dpf.translate(-dpf.get_center())

ev1,ev2=project_pcl_pca(dpf,
             voxel_size=0.02,
             nb=5,
             std_ratio=1.5,
             p_th=0.1,
             ig="Linearity",
             upper=True
            )

o3d.visualization.draw_geometries([dpf])

[]
Linearity


ValueError: cannot call `vectorize` on size 0 inputs unless `otypes` is set

In [18]:
print(dpf.get_min_bound())
dpf=o3d.io.read_point_cloud(os.path.join(data_dir,pcls[0]))
dpf.translate(-dpf.get_center())

maxp=np.max(np.hstack((dpf.get_min_bound(),dpf.get_max_bound())))
dpf.scale(1/maxp,center=dpf.get_center())

maxp=np.max(np.hstack((dpf.get_min_bound(),dpf.get_max_bound())))


[ 8874.2881355  34576.17243     9168.98617214]
803.0024390242306
0.9999999998369747


In [ ]:
#maxValorNubeOriginal=1035.978

In [21]:
data_dir="D:\\Documentos\\INNOVATE\\Base_de_datos\\BEAMS\\Aceptado"
pcls=os.listdir(data_dir)
os.path.join(data_dir,pcls[0])
_dir="D:\\Documentos\\INNOVATE\\Base_de_datos\\Debug\\seccion3.pts"
pcl=o3d.io.read_point_cloud(_dir)
pcl.translate(-pcl.get_center())
o3d.visualization.draw_geometries([pcl])

In [22]:
print(pcl.get_min_bound())
print(pcl.get_max_bound())

[-425.63947798 -259.02045024  -45.43556281]
[370.36052202 279.98114976  51.56443719]
